In [ ]:
import numpy as np
import pandas as pd
from sktime.forecasting.sarimax import SARIMAX
from sktime.split import ExpandingWindowSplitter
from sktime.performance_metrics.forecasting import MeanSquaredError
from sktime.forecasting.model_evaluation import evaluate

def main():
    # Simulated dataset
    np.random.seed(42)
    date_range = pd.date_range(start="2024-01-01", end="2025-12-31")
    data = {
        "feature_1": np.random.randn(len(date_range)),
        "feature_2": np.random.randn(len(date_range)),
        "target": np.random.randn(len(date_range)),
    }
    df = pd.DataFrame(data, index=date_range)
    df.index.name = "date"

    # Split data into train and test sets
    X = df[["feature_1", "feature_2"]]
    y = df[["target"]]
    X_train, X_test = X.loc["2024"], X.loc["2025"]
    y_train, y_test = y.loc["2024"], y.loc["2025"]

    

    # Debugging information
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_test shape: {y_test.shape}")
    print(f"X_test index: {X_test.index}")
    print(f"y_test index: {y_test.index}")

    # Simulate feature selection
    selected_features = np.array(["feature_1", "feature_2"])  # Change this to reproduce the issue
    print(f"Selected features: {selected_features}")

    

    # Ensure indices are aligned
    X_test, y_test = X_test.align(y_test, join="inner", axis=0)

    # Ensure X_train and X_test are DataFrames
    if not isinstance(X_train, pd.DataFrame):
        print("Error: X_train is not a DataFrame.")
        return
    if not isinstance(X_test, pd.DataFrame):
        print("Error: X_test is not a DataFrame.")
        return
    
    # Fit SARIMAX model
    forecaster = SARIMAX(order=(1, 1, 0), seasonal_order=None, freq="D")
    model = forecaster.fit(X=X_train.loc[:,selected_features], y=y_train)

    # Perform predictions
    cv = ExpandingWindowSplitter(initial_window=30, step_length=30, fh=[1])
    try:
        y_pred = model.update_predict(X= X_test.loc[:, selected_features], y=y_test, cv=cv, update_params=False, reset_forecaster=False)
        print(f"Predictions:\n{y_pred}")
    except Exception as e:
        print(f"Error during prediction: {e}")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'sktime'